<br></br>
# **gensim | doc2vec**
<br></br>
## **1 네이버 리뷰 문장의 활용**
네이버 영화리뷰 단어모델 만들기

In [ ]:
from konlpy.tag import Twitter
twitter = Twitter()

def read_data(filename):
    with open(filename, 'r') as f:
        data = [line.split('\t') for line in f.read().splitlines()]
        
    from random import randint
    random_data = [data[randint(1, len(data))]  for no in range(int(len(data)/10)) ]
    return random_data

def tokenize(doc):
    return ['/'.join(t) for t in twitter.pos(doc, norm=True, stem=True)]

In [ ]:
%%time
from collections import namedtuple
train_data        = read_data('./data/ratings_train.txt')
train_docs        = [(tokenize(row[1]), row[2]) for row in train_data[1:]]
TaggedDocument    = namedtuple('TaggedDocument', 'words tags')
tagged_train_docs = [TaggedDocument(d, [c]) for d, c in train_docs]

In [ ]:
from pprint import pprint
pprint(tagged_train_docs[0])

<br></br>
## **2 doc2vec 파라미터 설정 및 학습**

In [ ]:
model = gensim.models.doc2vec.Doc2Vec(N_corpus+R_corpus, dm=1, vector_size=200, min_count=3, epochs=10, hs=0)
model.train(N_corpus+R_corpus, total_examples=model.corpus_count, epochs=model.epochs)

In [ ]:
%%time
from gensim.models import doc2vec
doc_vectorizer = doc2vec.Doc2Vec(vector_size=300, alpha=0.025, min_alpha=0.025, seed=1234)
doc_vectorizer.build_vocab(tagged_train_docs)

for epoch in range(10):
    doc_vectorizer.train(tagged_train_docs, 
                         total_examples = doc_vectorizer.corpus_count, 
                         epochs = doc_vectorizer.epochs)
    doc_vectorizer.alpha -= 0.002
    doc_vectorizer.min_alpha = doc_vectorizer.alpha 

# 학습이 완료된 모델의 데이터를 저장한다
doc_vectorizer.save('')

<br></br>
## **3 doc2Vec 모델활용**

In [ ]:
from gensim.models import doc2vec
from pprint import pprint
doc_vectorizer = doc2vec.Doc2Vec.load('')

In [ ]:
pprint(doc_vectorizer.wv.most_similar('공포/Noun'))

In [ ]:
doc_vectorizer.wv.similarity('공포/Noun', 'ㅋㅋ/KoreanParticle')

In [ ]:
pprint(doc_vectorizer.wv.most_similar(positive=['여자/Noun', '공포/Noun'], 
                                      negative=['남자/Noun']))

In [ ]:
doc_vectorizer.infer_vector(['픽사/Noun', '최고/Noun', '명작/Noun'])[:10]

In [ ]:
doc_vectorizer.infer_vector(['픽사/Noun', '최고/Noun', '명작/Noun']).sum()

In [ ]:
doc_vectorizer.infer_vector(['호러/Noun', '여자/Noun', '공포/Noun'])[:10]

In [ ]:
doc_vectorizer.infer_vector(['호러/Noun', '여자/Noun', '공포/Noun']).sum()